# 面板数据分析
---

## 跨时横截面的混合

### 独立混合横截面概述

**独立混合横截面数据**

- 它是在不同时点从一个大总体里进行随机抽样的结果。

- 它们都是由独立抽取的观测所构成。

- 它与一个随机样本的差异在于，在不同时点上对总体进行抽样很可能导致观测点不是同分布的。

**面板数据**

- 要收集面板数据，要在不同时间跟踪相同的个人、家庭、企业或城市或别的什么单位。

- 就面板数据的计量经济分析而言，我们不能假定，不同时点的观测是独立分布的。

**关于独立混合横截面**

- 使用独立混合横截面的一个理由是要加大样本容量。

- 但仅当因变量和某些自变量保持着不随时间而变量的关系时，混合才是有用的。

- 为了反映总体在不同时期会有不同的分布，可以允许截距在不同时期有不同的值。

<br>

**例子：教育回报和工资中性别差异的变化**

In [1]:
library(rio)

cps <- import("http://fmwww.bc.edu/ec-p/data/wooldridge/cps78_85.dta")

In [2]:
# Detailed OLS results including interaction terms
summary(lm(lwage ~ y85*(educ+female) +exper+ I((exper^2)/100) + union, data=cps))


Call:
lm(formula = lwage ~ y85 * (educ + female) + exper + I((exper^2)/100) + 
    union, data = cps)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.56098 -0.25828  0.00864  0.26571  2.11669 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)    
(Intercept)       0.458933   0.093449   4.911 1.05e-06 ***
y85               0.117806   0.123782   0.952   0.3415    
educ              0.074721   0.006676  11.192  < 2e-16 ***
female           -0.316709   0.036621  -8.648  < 2e-16 ***
exper             0.029584   0.003567   8.293 3.27e-16 ***
I((exper^2)/100) -0.039943   0.007754  -5.151 3.08e-07 ***
union             0.202132   0.030294   6.672 4.03e-11 ***
y85:educ          0.018461   0.009354   1.974   0.0487 *  
y85:female        0.085052   0.051309   1.658   0.0977 .  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.4127 on 1075 degrees of freedom
Multiple R-squared:  0.4262,	Adjusted R-squared:  0.4219 
F-s

### 利用混合横截面做政策分析

当某些外生事件（常常是政府的政策改变）改变了个人、家庭、企业或城市运行的环境时，便产生了自然实验。一个自然实验总有一个不受政策变化影响的对照(control)组和一个被认为受政策变化影响的处理(treatment)组。为了控制好对照组和处理组之间的系统差异，我们需要两个年份的数据，一个在政策改变以前，另一个在政策改变之后。

$$y=\beta_{0}+\delta_{0}d2+\beta_{1}dT+\delta_{1}d2 \cdot dT + \mu$$

其中，$\hat{\delta}_{1}$是倍差估计量

$$\hat{\delta}_{1} = (\bar{y}_{2,T} - \bar{y}_{2,C}) - (\bar{y}_{1,T} - \bar{y}_{1,C})$$

<br>

**例子：垃圾焚化炉的区位对住房价格的影响**

In [12]:
library(stargazer)
library(tidyverse)

kielmc <- import("http://fmwww.bc.edu/ec-p/data/wooldridge/kielmc.dta")

-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

√ ggplot2 3.3.0     √ purrr   0.3.3
√ tibble  3.0.0     √ dplyr   0.8.5
√ tidyr   1.0.2     √ stringr 1.4.0
√ readr   1.3.1     √ forcats 0.5.0

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



In [24]:
kielmc_1981 <- dplyr::filter(kielmc, year > 1978)
kielmc_1978 <- dplyr::filter(kielmc, year < 1981)

ols_1981 <- lm(rprice~ nearinc, data=kielmc_1981)
ols_1978 <- lm(rprice~ nearinc, data=kielmc_1978)
ols_DID <- lm(rprice~ nearinc*y81, data=kielmc)
ols_DID_with_control <- lm(rprice~ nearinc*y81 + age + agesq + intst + land + area + rooms + baths, data=kielmc)

stargazer(ols_1981, ols_1978, ols_DID, ols_DID_with_control, type = "text", omit.stat = c("LL","ser","f"), digits = 2)


                               Dependent variable:                  
             -------------------------------------------------------
                                     rprice                         
                  (1)           (2)           (3)           (4)     
--------------------------------------------------------------------
nearinc      -30,688.27*** -18,824.37*** -18,824.37***   3,780.33   
              (5,827.71)    (4,744.59)    (4,875.32)    (4,453.42)  
                                                                    
y81                                      18,790.29***  13,928.48*** 
                                          (4,050.06)    (2,798.75)  
                                                                    
age                                                     -739.45***  
                                                         (131.13)   
                                                                    
agesq                            

## 面板数据模型

### 简单面板数据方法

利用面板数据，可以把影响因变量的无法观测因素分为两类：一类是恒常不变的，另一类则随时间而变化。我们可将含有单个可观测解释变量的两时期面板数据模型写成

$$y_{it} = \beta_{0} + \delta_{0}d2_{t} + \beta_{1}x_{it} + \alpha_{i} + \mu_{it},t=1,2$$

其中$\alpha_{i}$一般被称为非观测效应或固定效应，误差$\mu_{it}$常被称为特异性误差或时变误差。

<br>

**例子：城市犯罪率**

$$crmrte_{it} = \beta_{0} + \delta_{0}d87 + \beta_{1}unem_{it} + \alpha_{i} + \mu_{it}$$

<br>

#### 一阶差分估计量

在大多数应用中，收集面板数据的主要理由是为了考虑非观测效应$\alpha_{i}$与解释变量相关。

$$\Delta y_{i} = \delta_{0} + \beta_{1}\Delta x_{i} + \Delta \mu_{i}$$

我们用OLS估计上式，由此得到的估计量称为\textbf{一阶差分估计量}(first-differenced estimator)。这里有两个重要的假定，其一是$\Delta x_{i}$和$\Delta \mu_{i}$无关；其二是$\Delta x_{i}$必须因$i$的不同而有所变化。后者也很容易理解，由于我们容忍$\alpha_{i}$与$x_{it}$相关，所以我们就不要指望能把$\alpha_{i}$对$y_{it}$的影响与不随时间而变的任何变量的影响分离开来。

<br>

**例子：睡眠与工作的比较**

被解释变量是以分钟计的每周睡觉总时间，解释变量$totwrk$是每周总工作时间,$educ$是受教育年数，$marr$是婚姻的虚拟变量，$yngkid$是用以表示有无年幼子女的虚拟变量，$gdhlth$是“健康良好”虚拟变量。

In [25]:
slp75_81 <- import("http://fmwww.bc.edu/ec-p/data/wooldridge/slp75_81.dta")

In [26]:
summary(lm(cslpnap ~ ctotwrk + ceduc + cmarr + cyngkid + cgdhlth, data=slp75_81))


Call:
lm(formula = cslpnap ~ ctotwrk + ceduc + cmarr + cyngkid + cgdhlth, 
    data = slp75_81)

Residuals:
    Min      1Q  Median      3Q     Max 
-2454.2  -307.2    79.8   334.4  2037.9 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) -92.63404   45.86590  -2.020   0.0446 *  
ctotwrk      -0.22667    0.03605  -6.287 1.58e-09 ***
ceduc        -0.02447   48.75938  -0.001   0.9996    
cmarr       104.21395   92.85536   1.122   0.2629    
cyngkid      94.66540   87.65252   1.080   0.2813    
cgdhlth      87.57785   76.59913   1.143   0.2541    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 598.6 on 233 degrees of freedom
Multiple R-squared:  0.1495,	Adjusted R-squared:  0.1313 
F-statistic: 8.191 on 5 and 233 DF,  p-value: 3.827e-07


#### 用两期面板数据做政策分析

令$y_{it}$为结果变量，并令$prog_{it}$为项目参与虚拟变量，最简单的非观测效应模型为

$$y_{it} = \beta_{0} + \delta_{0}d2_{t} + \beta_{1}prog_{it} + \alpha_{i} + \mu_{it},t=1,2$$

如果项目参与仅发生在第二个时期，那么在差分方程中$\beta_{1}$的OLS估计量就有一个非常简单的表达式

$$\hat{\beta}_{1} = \bar{\Delta y}_{treat} - \bar{\Delta y}_{control}$$

即我们计算处理组和对照组在这两个时期的平均变化，然后取两者之差便是$\hat{\beta}_{1}$。

<br>

#### 多于两期的差分法

三期面板数据的一阶差分方程

$$\Delta y_{it} = \alpha_{0} + \alpha_{3}d3_{t} + \beta_{1}\Delta x_{it1} + ... + \beta_{k}\Delta x_{itk} + \Delta \mu_{it},t=2,3$$

多于三个时期面板数据的一阶差分方程

$$\Delta y_{it} = \alpha_{0} + \alpha_{3}d3_{t} + ... + \alpha_{T}T_{t} + \beta_{1}\Delta x_{it1} + ... + \beta_{k}\Delta x_{itk} + \Delta \mu_{it},t=2,...,T$$

如果我们使用多于两期的数据时，欲使用通常的标准误和检验统计量恰当，我们必须假定$\Delta_{it}$是序列无关的。事实上，若假定$\mu_{it}$序列无关且具有恒定方程，则可以证明$\Delta \mu_{i,t}$与$\Delta \mu_{i,t+1}$之间的相关系数为$-0.5$。若$\mu_{it}$遵循一个稳定的AR(1)模型，则$\mu_{it}$将是序列相关的。只有当$\mu_{it}$遵循一个随机游走时，$\Delta \mu_{it}$才是序列无关的。

### 固定效应和随机效应

#### 固定效应估计法

在某些假定下，起到更好作用的另一种方法是所谓的固定效应变换。考虑仅有一个解释变量的模型，有

$$y_{it} = \beta_{1}x_{it} + \alpha_{i} + \mu_{it},t=1,2,...,T$$

现在对每个$i$求方程在时间上的平均，便得到

$$\bar{y}_{i} = \beta_{1}\bar{x}_{i} + \alpha_{i} + \bar{\mu}_{i}$$

用第一个式子减去第二个式子，得到

$$y_{it} - \bar{y}_{i} = \beta_{1}(x_{it} - \bar{x}_{i}) + (\mu_{it} - \bar{\mu}_{i}),t=1,2,...,T$$

或

$$\ddot{y}_{it} = \beta_{1}\ddot{x}_{it} + \ddot{\mu}_{it},t=1,2,...T$$

对上述式子进行回归得到的混合OLS估计量被称为固定效应估计量或组内估计量。

**关于固定效应估计量**

- 在解释变量的严格外生性假定下，固定效应估计量是无偏的

- 凡是在时间上恒定的解释变量都必定随固定效应变换而消失，因此我们不能包含诸如性别或城市的地理位置等变量

- 如果把全部年度虚拟变量包含进来时，对于任何在时间上的变化为常数的变量，都无法估计其影响。

- 为了使得OLS分析有效，还需要的假定是误差$\mu_{it}$的同方差性与无序列相关性

- 当我们使用混合OLS估计除时间均值的方程时，我们总共有$NT$个观测值和$k$个自变量。对每个横截面观测$i$，由于取时间平均，会失去一个自由度。因此，恰当的自由度是$df=NT-N-k=N(T-1)-k$。如果我们需要自己来做除时间均值运算且用混合OLS进行估计，就有必要对标准误和$t$统计量加以修正。

- 在做了固定效应之后，要计算$\hat{\alpha}_{i}$相当容易，即$\hat{\alpha}_{i} = \bar{y}_{i} - \hat{\beta}_{1}\bar{x}_{i1} - ... - \hat{\beta}_{k}\bar{x}_{ik},i = 1,2,...,N$

- 在固定效应(FE)估计中所报告的截距是$\hat{\alpha}_{i}$在$i$上的平均值。随着$T$变大，我们可以得到$\alpha_{i}$更好的估计值，但大多数面板数据集都属于$N$大而$T$小的类型。

<br>

**例子：废弃率方程的固定效应估计**

In [32]:
library(plm)

jtrain <- import("http://fmwww.bc.edu/ec-p/data/wooldridge/jtrain.dta")
jtrain <- pdata.frame(jtrain, index=c("fcode", "year"))

In [33]:
summary(plm(lscrap ~ d88 + d89 + grant + grant_1 + lsales + lemploy, data=jtrain, model="within"))

Oneway (individual) effect Within Model

Call:
plm(formula = lscrap ~ d88 + d89 + grant + grant_1 + lsales + 
    lemploy, data = jtrain, model = "within")

Unbalanced Panel: n = 51, T = 1-3, N = 148

Residuals:
     Min.   1st Qu.    Median   3rd Qu.      Max. 
-1.858662 -0.133751 -0.021484  0.148632  1.580473 

Coefficients:
          Estimate Std. Error t-value Pr(>|t|)  
d88     -0.0039605  0.1195487 -0.0331  0.97364  
d89     -0.1321925  0.1536863 -0.8601  0.39197  
grant   -0.2967544  0.1570860 -1.8891  0.06206 .
grant_1 -0.5355783  0.2242060 -2.3888  0.01897 *
lsales  -0.0868607  0.2596993 -0.3345  0.73880  
lemploy -0.0763642  0.3502912 -0.2180  0.82792  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Total Sum of Squares:    27.934
Residual Sum of Squares: 21.982
R-Squared:      0.21306
Adj. R-Squared: -0.27121
F-statistic: 4.10625 on 6 and 91 DF, p-value: 0.0010818

利用$54$个企业在$1987$年、$1988$年和$1989$年报告的每年废弃率数据。在$1988$年以前没有任何企业享受培训津贴，到$1988$年有$19$个企业获得津贴；而在$1989$年又有$10$个不同的企业获得津贴。结果发现，1988年获得津贴的企业将于1989年降低其废弃率的$34.4\%[exp(-0.422)-1 \approx -3.44]$

#### 虚拟变量回归

对每个$i$估计一个截距的方法，是连同解释变量一起，给每一个横截面观测单位安排一个虚拟变量，这一方法被称为虚拟变量回归。

**虚拟变量回归的特点**

- 它给出的$\beta_{j}$估计值与固定效应估计值恰好一样，且标准误和其他主要统计量也是一样。

- 做虚拟变量回归的一个好处是，可以直接算出恰当的自由度

- 虚拟变量回归算出的$R^{2}$通常都比较高，因此对每个横截面单位都包含了一个虚拟变量，以致能解释数据中变化的大部分。

- 可以直接得到截距$\hat{\alpha}_{i}$

<br>

#### 固定效应还是一阶差分

- 在$T=2$时，FE和FD的估计值及其全部检验统计量都完全一样。

- 在$T \geq 3$时，在FE假定下，两者都是无偏且一致的。对较大的$N$和较小的$T$，FE和FD之间的选择关键在其估计量的相对效率。当$\mu_{it}$无序列相关时，固定效应法比一阶差分更有效。当$\mu_{it}$遵循随机游走，那么一阶差分法更好。在许多情形下，$\mu_{it}$表现出某种正的序列相关，却未必达到一个随机游走的程度，这时要比较FE和FD估计量的效率就不那么容易了。

- 将这两种方法都试下常常是一个好主意：如果结果都差不多，也就无所谓了。

<br>

#### 非平衡面板数据

在一些面板数据集中，样本缺少了某些横截面单位的某些年份数据，这些数据集为非平衡面板数据。如果一些$i$之所以缺失数据的理由与特异误差$\mu_{it}$无关，非平衡面板数据就不会引发什么问题。当我们的数据是关于个人、家庭或企业时，事情往往比较复杂。

**非平衡面板数据的固定效应法**

设$T_{i}$为横截面单位$i$的时期数，我们只需用$T_{i}$个观测去做除时间均值的运算。观测总数将是$T_{1}+T_{2}+...+T_{N}$。

<br>

#### 随机效应模型

假定一个非观测效应模型

$$y_{it} = \beta_{0} + \beta_{1}x_{it1} + ... + \beta_{k}x_{itk} + \alpha_{i} + \mu_{it}$$

如果假定非观测效应$\alpha_{i}$与每个解释变量都无关

$$Cov(x_{itj},\alpha_{i}) = 0, t=1,2,..,T; j = 1,2,...,k$$

上述方程就成为一个随机效应模型。 如果按照混合OLS估计此方程，这样做也能得到$\beta_{j}$的一致估计量，但它忽略了模型的基本要点。

定义复合误差项$\nu_{it} = \alpha_{i} + \mu_{it}$，则

$$Corr(\nu_{it},\nu_{is}) = \sigma_{\alpha}^{2}/(\sigma_{\alpha}^{2} + \sigma_{\mu}^{2}),t\neq s$$

误差项这种正的序列相关可能很大，由于混合OLS标准误忽视了这种相关，所以不正确。可以通过GLS变换以消去误差中的序列相关，定义

$$\theta = 1 - [\sigma_{\mu}^{2}/(\sigma_{\mu}^{2} + T \sigma_{\alpha}^{2})]^{1/2}$$

它介于$0$与$1$之间。变换后的方程是

$$y_{it} - \theta \bar{y}_{i} = \beta_{0}(1-\theta) + \beta_{1}(x_{it1} - \theta \bar{x}_{i1}) + ... + \beta_{k}(x_{itk} - \theta \bar{x}_{ik}) + (\nu_{it} - \theta \bar{\nu}_{i})$$

它使用每个变量的准除均值数据。由于$\theta$是未知的，但是可以估计。用$\hat{\theta}$代替$\theta$的可行GLS估计量被称为随机效应估计量。

**随机效应估计量的性质**

- 它容许我们考虑不随时间而变化的解释变量

- 它是一致的，但不是无偏的

- 当$\theta=0$时，得到混合OLS，而当$\theta=1$时则得到FE。当非观测效应$\alpha_{i}$相对不重要时，RE估计量接近混合OLS，反之则接近FE。

**准除均值误差**

$$\nu_{it} - \theta \bar{\nu}_{i} = (1-\theta)\alpha_{i} - \mu_{it} - \theta \bar{\mu}_{i}$$

当$\theta \rightarrow 1$，偏误项趋于零，RE估计量趋于FE估计量。

In [36]:
wagepan <- import("http://fmwww.bc.edu/ec-p/data/wooldridge/wagepan.dta")
wagepan_pd <- pdata.frame(wagepan, index=c("nr", "year"))

In [39]:
formula <- lwage ~ educ + black + hisp + exper + expersq + married + union + d81 + d82 + d83 + d84 + d85 + d86 + d87
ols_model <- lm(formula, data=wagepan)
re_model <- plm(formula, data=wagepan_pd, model="random")
fe_model <- plm(lwage ~ educ + black + hisp + expersq + married + union + d81 + d82 + d83 + d84 + d85 + d86 + d87, data=wagepan_pd, model="within")

stargazer(ols_model, re_model, fe_model, type = "text", omit.stat = c("LL","ser","f"), digits = 4)


                   Dependent variable:       
             --------------------------------
                          lwage              
                OLS             panel        
                               linear        
                (1)        (2)        (3)    
---------------------------------------------
educ         0.0913***  0.0919***            
              (0.0052)   (0.0107)            
                                             
black        -0.1392*** -0.1394***           
              (0.0236)   (0.0477)            
                                             
hisp           0.0160     0.0217             
              (0.0208)   (0.0426)            
                                             
exper        0.0672***  0.1058***            
              (0.0137)   (0.0154)            
                                             
expersq      -0.0024*** -0.0047*** -0.0052***
              (0.0008)   (0.0007)   (0.0007) 
                                 

#### 随机效应还是固定效应

由于固定效应容许非观测效应与解释变量相关，而随机效应则不然，普遍认为FE是更令人信服的工具。随机效应在某些特定情形中仍可适用。最明显的是，若关键解释变量不随着时间而变化，我们就不能用FE估计其对$y$的影响。

**豪斯曼检验**

相当常见的是，研究者同时适用随机效应和固定效应，然后规范地检验时变解释变量系数的统计显著差别。

### 相关随机效应方法

在某些应用中，我们可以把$\alpha_{i}$合理地当作随机变量应用。假设$\alpha_{i}$与$x_{it}$的平均水平相关，则

$$\alpha_{i} = \alpha + \gamma \bar{x}_{i} + r_{i}$$

其中，假设$r_{i}$与每个$x_{it}$都不相关，即$Cov(\bar{x}_{i},r_{i})=0$。

运用相关随机效应(CRE)方法，得到

$$y_{it} = \beta x_{it} + \alpha + \gamma \bar{x}_{i} + r_{i} + \mu_{it} = \alpha + \beta x_{it} + \gamma \bar{x}_{i} + r_{i} + \mu_{it} $$

对上式进行RE回归分析，得到CRE估计值。可得到$\hat{\beta}_{CRE} = \hat{\beta}_{FE}$。考虑CRE的理由起码有两个，其一，CRE方法提供了一个简单、正式地选择FE和RE的方法；其二是它提供了在固定效应分析中包含不随时间变化解释变量的一个途径，例如$y_{it} = \alpha + \beta x_{it} + \gamma \bar{x}_{i} + \delta z_{i} + r_{i} + \mu_{it}$。

### 面板数据方法用于其他的数据结构

各种面板数据方法都可以用于一些不涉及时间的数据结构。例如，在人口统计学中，常用兄弟姐妹去控制观测不到的家庭背景特征。通常我们希望容许无法观测的“家庭效应”与可以观测的解释变量相关。如果那些解释变量在一个家庭内的兄弟姐妹之间有所变化，那么在两兄弟或两姐妹之间取差分（或更一般地利用一个家庭内部的组内变换）就是一种可取的估计方法。通过消除非观测效应，我们就消除了由复杂的家庭背景特征导致的潜在偏误。

** 例子：成对的姐妹数据研究未成年生育对未来经济状态造成的影响**

$$\begin{aligned} \log \left(\text {incneeds}_{f s}\right)=& \beta_{0}+\delta_{0} \operatorname{sister} 2_{s}+\beta_{1} \text { teenbrth }_{f s} \\ &+\beta_{2} a g e_{f s}+\text { other factors }+a_{f}+u_{f s} \end{aligned}$$

其中，$f$指家庭，而$s$指该家庭中的姐妹之一。姐姐的截距是$\beta_{0}$，妹妹的是$\beta_{0}+\delta_{0}$，所关注的变量是$teenbrth_{fs}$，这是一个二值变量，当家庭$f$的姐妹$s$在十几岁时生育小孩，它就等于$1$，变量$age_{fs}$是家庭$f$中姐妹$s$的当时年龄。估计方法：在每个家庭之内，在姐妹之间取差分，于是得到

$$\Delta log(incneeds)=\delta_{0}+\beta_{1}\Delta teenbrth + \beta_{2}\Delta age + ... + \Delta \mu$$